In [1]:
import numpy as np
import autokeras as ak
from ReadData import read_data_as_img
from Models import lenet, alexnet, widenet
from keras import backend as K
from keras import Sequential
from keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [7]:
seed = 42
np.random.seed(seed)

data_dir = "../data"

In [3]:
X, y = read_data_as_img(data_dir, "OPN")
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
K.set_image_data_format("channels_last")

In [11]:
model = Sequential(load_model("buen_modelo/best_model").layers)
model.compile(optimizer="adam", loss=BinaryCrossentropy(), metrics=["accuracy"])
# Evaluate model
score = model.evaluate(X_test, y_test, batch_size=128)
print("Test loss, Test accuracy", score)

14/14 [==============================] - 4s 294ms/step - loss: 0.0731 - accuracy: 0.9936
Test loss, Test accuracy [0.07305172830820084, 0.9936158061027527]


In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 200, 200, 1)       3         
_________________________________________________________________
conv2d (Conv2D)              (None, 198, 198, 16)      160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 196, 64)      9280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 98, 98, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 614656)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 614656)           

In [13]:
y_test_pred = model.predict(X_test)
y_test_pred = y_test_pred.round().astype(int)

In [14]:
print(f"Predicciones clase 0: {sum(y_test_pred.round().astype(int) == 0).sum()}/{y_test_pred.shape[0]}")
print(f"Predicciones clase 1: {sum(y_test_pred.round().astype(int) == 1).sum()}/{y_test_pred.shape[0]}")

Predicciones clase 0: 1570/1723
Predicciones clase 1: 153/1723


# F1 score

In [15]:
from sklearn.metrics import f1_score

In [16]:
f1_score(y_test, y_test_pred)

0.9648562300319489

# Binary Crossentropy
No es lo mejor en desbalanceados


In [17]:
from sklearn.metrics import log_loss 

In [18]:
log_loss(y_test, y_test_pred, eps=1e-15)

0.22050385347584417

# AUC

In [26]:
import numpy as np
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_test_pred)

0.971235204734485

# Accuracy


In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_test_pred)

0.9936157864190366